In [2]:
import pandas as pd
import os
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import MinMaxScaler

import wrangle as w
import explore as ex

In [4]:
df=w.get_amzn()
train,test=w.train_test_split(df)

In [5]:
len(train), len(test)

(4398, 1885)

In [ ]:
def arima_model(x_train, y_train, order=(1, 1, 1), scale=True):
    scaler = MinMaxScaler()
    x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns, index=x_train.index)

    # Combine features and target variable
    train_data = pd.concat([x_train_scaled, y_train], axis=1)

    # Fit ARIMA model
    model = ARIMA(train_data[y_train.name], order=order, trend='c')  # 'c' adds a constant term
    model_fit = model.fit()

    # Make predictions on the training set
    y_train_pred = model_fit.predict()

    # Calculate accuracy and recall
    ytr_acc = accuracy_score(y_train, y_train_pred.round())
    tr_rec = recall_score(y_train, y_train_pred.round())

    # Print accuracy and recall
    print(f'Train Accuracy:      {round(ytr_acc*100,2)}%')
    print(f'Train Recall:        {round(tr_rec*100,2)}%')

    return model_fit, y_train_pred, ytr_acc, tr_rec

# Example usage:
# Assuming x_train and y_train are your training features and target variable
# model, y_train_pred, ytr_acc, tr_rec = arima_model(x_train, y_train, order=(1, 1, 1), scale=True)
